In [1]:
import pandas as pd 
import numpy as np
from tabulate import tabulate
data = pd.read_csv("./outputs/blocks-world.csv")

In [2]:
data.head(5)

,Percent,Real_Goal,Time,Cost,Prob,Results
0,10,0,232570019,32/32/31/31/32/31/32/32/32/31/31/32/32/32/31/3...,0.04697991789983241/0.04697991789983241/0.0484...,2/3/5/9/10/14/15/18/19/
1,10,1,65295083,32/31/32/31/32/32/32/32/32/32/32/32/32/32/32/3...,0.04740408089661592/0.04890884795363763/0.0474...,1/3/18/
2,10,2,83097638,32/32/31/31/32/31/32/32/32/31/31/32/32/32/31/3...,0.04697991789983241/0.04697991789983241/0.0484...,2/3/5/9/10/14/15/18/19/
3,10,5,60948554,31/31/31/31/32/32/31/31/31/32/32/31/32/32/31/3...,0.04825533066642492/0.04825533066642492/0.0482...,0/1/2/3/6/7/8/11/14/18/19/20/
4,10,5,68227266,31/31/31/31/32/32/31/31/31/32/32/31/32/32/31/3...,0.04825533066642492/0.04825533066642492/0.0482...,0/1/2/3/6/7/8/11/14/18/19/20/


In [3]:
def precision(stringList, answer):
    goal_count = 0
    found = 0
    for result in stringList:
        if result == str(answer):
            found = 1
        goal_count += 1
    return found/(goal_count-1)

def recall(stringList, answer):
    found = 0
    for result in stringList:
        if result == str(answer):
            found = 1
            break
    return found

def accuracy(total, stringList, answer):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for result in stringList[0:-1]:
        if result == str(answer):
            tp += 1
        else:
            fp += 1
    
    fn = 1 - tp
    
    # total is the number of all goals
    tn = total - tp - fp - fn
    return (tp + tn)/(tn + tp + fp + fn)

def func_precision(stringList, answer):
    goal_count = 0
    found = 0
    for result in stringList:
        if result == str(answer):
            found = 1
        goal_count += 1
    return found/(goal_count-1)

def func_recall(stringList, answer):
    found = 0
    for result in stringList:
        if result == str(answer):
            found = 1
            break
    return found

def func_accuracy(total, stringList, answer):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for result in stringList[0:-1]:
        if result == str(answer):
            tp += 1
        else:
            fp += 1
    
    fn = 1 - tp
    
    # total is the number of all goals
    tn = total - tp - fp - fn
    return (tp + tn)/(tn + tp + fp + fn)


In [27]:
rows = data.loc[data['Percent'] == 30]
def calculate_statistics(rows):
    length = rows.shape[0]
    
    precision = 0
    recall = 0
    accuracy = 0
        
    for index, row in rows.iterrows():
        
        answer = row["Real_Goal"]
        results = row["Results"].split("/")
        all_candidates = row["Cost"].split("/")
        
        total = len(all_candidates)-1   # the last one is /
        
        
        
        p = func_precision(results, answer)
        r = func_recall(results, answer)
        a = func_accuracy(total, results, answer)
        
        precision += p
        recall += r
        accuracy += a
        
    precision = precision/length
    recall = recall/length
    accuracy = accuracy/length 
    
    return precision, recall, accuracy

p,r,a = calculate_statistics(rows)

print(p)
print(r)
print(a)

0.2142559425515511
0.7723577235772358
0.6191153697251277


In [28]:
length = data.shape[0]
length

1076

In [29]:
percent = 10

matrix = []
temp_storage = [0,0,0,0] # p, r, accuracy, time
num = 0

count_goals = 0

for index in range(length):
    new_percent = data.loc[index, "Percent"]
    answer = data.loc[index, "Real_Goal"]
    results = data.loc[index, "Results"].split("/")
    all_candidates = data.loc[index, "Cost"].split("/")
    total = len(all_candidates)-1   # the last one is /
    p = precision(results, answer)
    r = recall(results, answer)
    a = accuracy(total, results, answer)
    
    count_goals += total
    
    time = data.loc[index, "Time"]
    num += 1
    
    if (percent != new_percent):
        temp_storage[0] = temp_storage[0]/num
        temp_storage[1] = temp_storage[1]/num
        temp_storage[2] = temp_storage[2]/num
        temp_storage[3] = temp_storage[3]/num
        
        matrix.append(temp_storage)
        temp_storage = [0,0,0,0]
        num = 0
        percent = new_percent
        
        
    temp_storage[0] += p
    temp_storage[1] += r
    temp_storage[2] += a
    temp_storage[3] += time
    
num += 1
temp_storage[0] = temp_storage[0]/num
temp_storage[1] = temp_storage[1]/num
temp_storage[2] = temp_storage[2]/num
temp_storage[3] = temp_storage[3]/num   
matrix.append(temp_storage)
    
    


In [30]:
table = tabulate(matrix, headers=["p", "r", "a", "t"])
print(table)

       p         r         a            t
--------  --------  --------  -----------
0.079582  0.611336  0.424022  2.80924e+07
0.214256  0.772358  0.619115  3.14734e+07
0.285467  0.784553  0.686498  3.76292e+07
0.505923  0.902439  0.762089  4.88199e+07
0.747283  1         0.822981  5.99323e+07


In [17]:
## export the table to csv
# A = np.array(matrix)
# np.savetxt("blocksworld.csv", A, delimiter=",")